In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta 
import sqlite3
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#mes = diciembre
#cierre mes = '20221231'
def Tabla_e(mes,cierre_mes):   
    #Lectura Datos
    tabla_e = pd.read_csv(r"..\Claro\Stocks mensuales\Stock HFC {}.csv".format(mes),encoding='latin-1',sep=';')
    tabla_e = tabla_e.loc[(tabla_e.ESTADO_SERVICIO == 'Activo')&(tabla_e.ESTADO_CLIENTE == 'Cliente')]
    dic = pd.read_excel(r"Datos-Tabla E CLARO\DICCIONARIO_V3.xlsb",sheet_name = 'TODO PLAN HOGAR', engine='pyxlsb')
    #correcion codigo plan actual
    y = tabla_e.loc[tabla_e.PLAN_NEW.isnull() == True]
    y.PLAN_NEW = y.PLAN_CONTRATADO.values
    y.CODIGO_PLAN_NEW = y.CODIGO_PLAN.values
    x =  tabla_e.loc[tabla_e.PLAN_NEW.isnull() != True]
    tabla_e = pd.concat([y,x]).sort_index()

    #procesamiento datos
    tabla_e= tabla_e.drop(columns=['PLAN_CONTRATADO', 'CODIGO_PLAN','FECHA_INSTALACION_SERVICIO'])
    tabla_e = tabla_e.merge(dic[['NOMBRE PLAN','PLAN ID','MONTO PLAN FINAL', 'PLAY','PRECIO_TV $', 'PRECIO_TLF $', 'PRECIOIBA $', 'MONTO MULTISERVICIO',
           'DSCTO_MULTISERVICIO_TV -$', 'DSCTO_MULTISERVICIO_TLF -$','DSCTO_MULTISERVICIO_IBA -$', 'DESCUENTO MULTISERVICIO','CLAVE_TV',
           'CLAVE_TLF', 'CLAVE_IBA', 'NOMBRE_TV', 'NOMBRE_TLF', 'NOMBRE_IBA','VELOCIDAD INTERNET','TIPO TECNOLOGÍA']], how = 'left', left_on = 'CODIGO_PLAN_NEW',right_on='PLAN ID')


    #creacion columnas de descuentos
    adq = pd.read_csv(r"..\Claro\INPUTS\Adquisicion\Adquisicion Activos\{} Descuentos Adquisicion Activos.csv".format(cierre_mes), delimiter = ';',encoding='latin-1')
    ret = pd.read_csv(r"..\Claro\INPUTS\Retencion\Retencion Activos\{} Descuentos Retencion Activos.csv".format(cierre_mes), delimiter = ';',encoding='latin-1')

    #renombrar columnas para que quede similar a la data de VTR
    tabla_e.rename(columns = {'PLAN_NEW':'NOMBRE_PROMOCION','CODIGO_PLAN_NEW':'ID_PROMO','PLAY':'N_EMPAQUETAMIENTO',
                             'PRECIO_TV $':'RENTA_CABLE','PRECIO_TLF $':'RENTA_FONO','PRECIOIBA $':'RENTA_INTERNET','DESCUENTO MULTISERVICIO':'DCTO_MULTISERVICIO',
                             'CLAVE_TV':'ID_CABLE','CLAVE_TLF':'ID_FONO','CLAVE_IBA':'ID_INTERNET'},inplace = True)

    adq1 = adq.groupby('ID_CLIENTE').agg(VALOR_DESC_ADQUISICION = ('MONTO_DSCTO','sum')).reset_index()
    adq1.rename(columns = {'ID_CLIENTE' : 'RUT'}, inplace = True)

    ret1 = ret.groupby('ID_CLIENTE').agg(VALOR_DESC_RETENCION = ('MONTO_DSCTO','sum')).reset_index()
    ret1.rename(columns = {'ID_CLIENTE' : 'RUT'}, inplace = True)


    # merge de columnas de descuento

    tabla_e = tabla_e.merge(adq1,how = 'left', on = ['RUT'])
    tabla_e = tabla_e.merge(ret1,how = 'left', on = ['RUT'])

    tabla_e['VALOR_DESC_ADQUISICION'] = tabla_e['VALOR_DESC_ADQUISICION'].fillna(0)
    tabla_e['VALOR_DESC_RETENCION'] = tabla_e['VALOR_DESC_RETENCION'].fillna(0)
    tabla_e['RENTA_PRINCIPAL'] = tabla_e['MONTO PLAN FINAL']
    tabla_e['RENTA_PRINCIPAL_2'] = tabla_e['RENTA_PRINCIPAL'] + tabla_e['VALOR_DESC_ADQUISICION'] + tabla_e['VALOR_DESC_RETENCION']
    tabla_e['N_RGU'] = np.where(tabla_e.N_EMPAQUETAMIENTO == '1 PLAY',1,
                            np.where(tabla_e.N_EMPAQUETAMIENTO == '2 PLAY',2,3))

    # Reemplazar '-' con NaN en las columnas relevantes
    columns_to_replace = ['ID_CABLE', 'ID_INTERNET', 'ID_FONO']
    tabla_e[columns_to_replace] = tabla_e[columns_to_replace].replace('-', np.nan)

    conditions_bundle = [((~tabla_e['ID_CABLE'].isnull()) & (tabla_e['ID_INTERNET'].isnull()) & (tabla_e['ID_FONO'].isnull())),
                       ((~tabla_e['ID_CABLE'].isnull()) & (tabla_e['ID_INTERNET'].isnull()) & (~tabla_e['ID_FONO'].isnull())),
                       ((~tabla_e['ID_CABLE'].isnull()) & (~tabla_e['ID_INTERNET'].isnull()) & (~tabla_e['ID_FONO'].isnull())),
                       ((~tabla_e['ID_CABLE'].isnull()) & (~tabla_e['ID_INTERNET'].isnull()) & (tabla_e['ID_FONO'].isnull())),
                       ((tabla_e['ID_CABLE'].isnull()) & (tabla_e['ID_INTERNET'].isnull()) & (~tabla_e['ID_FONO'].isnull())),
                       ((tabla_e['ID_CABLE'].isnull()) & (~tabla_e['ID_INTERNET'].isnull()) & (~tabla_e['ID_FONO'].isnull())),
                       ((tabla_e['ID_CABLE'].isnull()) & (~tabla_e['ID_INTERNET'].isnull()) & (tabla_e['ID_FONO'].isnull()))]
    choices_bundle = ['C', 'C + F', '3PLAY', 'C + I', 'F', 'I + F', 'I']
    choices_bundle2 = [1, 2, 3, 2, 1, 2, 1]
    tabla_e['BUNDLE'] = np.select(conditions_bundle, choices_bundle, default='-1')
    tabla_e['NUM_RGU'] = np.select(conditions_bundle, choices_bundle2, default= 0)

    tabla_e['DSCTO_MULTISERVICIO_TV -$'] = tabla_e['DSCTO_MULTISERVICIO_TV -$'].fillna(0)
    tabla_e['DSCTO_MULTISERVICIO_TLF -$'] = tabla_e['DSCTO_MULTISERVICIO_TLF -$'].fillna(0)
    tabla_e['DSCTO_MULTISERVICIO_IBA -$'] = tabla_e['DSCTO_MULTISERVICIO_IBA -$'].fillna(0)
    tabla_e['DCTO_MULTISERVICIO'] = tabla_e['DCTO_MULTISERVICIO'].fillna(0)

    tabla_e = tabla_e.loc[(tabla_e['NOMBRE PLAN'].isnull() == False)&(tabla_e.TIPO_CLIENTE == 'PERSONA')]
    #transformacion dtypes
    tabla_e = tabla_e.astype({"MONTO PLAN FINAL":"int","RENTA_CABLE":"int","RENTA_FONO":"int","RENTA_INTERNET":"int","MONTO MULTISERVICIO":"int","DSCTO_MULTISERVICIO_TV -$":"int",
                              "DSCTO_MULTISERVICIO_TLF -$":"int","DSCTO_MULTISERVICIO_IBA -$":"int","DCTO_MULTISERVICIO":"int","VALOR_DESC_ADQUISICION":"int",
                              "VALOR_DESC_RETENCION":"int","RENTA_PRINCIPAL":"int","RENTA_PRINCIPAL_2":"int",})
    tabla_e['RENTA_FONO1'] = tabla_e['RENTA_FONO'] - tabla_e['DSCTO_MULTISERVICIO_TLF -$']
    tabla_e['RENTA_CABLE1'] = tabla_e['RENTA_CABLE'] - tabla_e['DSCTO_MULTISERVICIO_TV -$']
    tabla_e['RENTA_INTERNET1'] = tabla_e['RENTA_INTERNET'] - tabla_e['DSCTO_MULTISERVICIO_IBA -$']
    return tabla_e

In [ ]:
tabla_e= Tabla_e('octubre 2023','20231031')

In [ ]:
tabla_e.to_csv(r"..\Claro\Tabla E Claro\202310_TablaE_Claro.csv",index = False)

In [ ]:
#Cruce con Planes Claro Video 

In [ ]:
# Carga la base de datos de Planes Claro Video desde el archivo Excel
planes_claro_video = pd.read_excel(r"C:\Users\wduran\CLAROCHILE\Nicoll Constanza Aguirre Diaz - dcabello\Wilder\MIgracion Andes-Sur\Planes Claro Video.xlsx")

# Carga la base de datos de Stock Clientes Claros desde el archivo CSV
stock_clientes_claros = pd.read_csv(r"C:\Users\wduran\CLAROCHILE\Nicoll Constanza Aguirre Diaz - dcabello\Claro\Tabla E Claro\202309_TablaE_Claro.csv",encoding='ISO-8859-1',sep=';')

# Realiza una fusión (merge) de ambas bases de datos en base al campo 'CODIGO_PLAN_NEW'
resultado = pd.merge(stock_clientes_claros, planes_claro_video, left_on='PLAN ID', right_on='CODIGO_PLAN_NEW', how='inner')

# Selecciona solo las columnas que deseas en el resultado
resultado = resultado[['RUT','CODIGO_PLAN_NEW' ,'PLAN_CON_CLAROV','RENTA_PRINCIPAL','VALOR_DESC_ADQUISICION','VALOR_DESC_RETENCION','RENTA_PRINCIPAL_2','MONTO_ULTIMO_PAGO','PAQUETE']]

# Guarda el resultado en un nuevo archivo Excel
resultado.to_excel('Stock_clientes_Claro_Video.xlsx', index=False)

In [ ]:
#version arreglada de bundle VTR

In [24]:
# Ruta de tu base de datos
ruta_base_datos = r"C:\Users\wduran\CLAROCHILE\Nicoll Constanza Aguirre Diaz - dcabello\Dashboards Nueva Oferta\INPUTS\Tablas E\20231031_StockTotal.csv"

# Leer la base de datos con el separador adecuado (coma)
df = pd.read_csv(ruta_base_datos, encoding='latin-1', sep=',')

# Imprimir las primeras filas del DataFrame para verificar las columnas
print(df.head())

      FECHA RUT_PERSONA  IDEN_VIVIENDA   NOMBRE_CTA_SERV ID_CABLE  \
0  20231031  12941753-6       19844292  12941753-6-S-001  1-5M4L6   
1  20231031   7860335-6        2963044   7860335-6-S-001  1-5M4L6   
2  20231031  19364705-7       19273347  19364705-7-S-006  1-5M4L6   
3  20231031   3360967-1       19468230   3360967-1-S-001  1-KT59J   
4  20231031   7958748-6       19521216   7958748-6-S-001  1-ANC1R   

                NOMBRE_PROMOCION  ID_PROMO                 NOMBRE_CABLE  \
0               VTR Plus 3 1122F  1-13AOTW           TV Full HD Digital   
1               VTR Plus 3 1122F  1-13AOTW           TV Full HD Digital   
2               VTR Plus 2 0822F  1-135UY4           TV Full HD Digital   
3  Triplepack Negocios Mas 1122F  1-13AQMH  TV Negocios Full HD Digital   
4             Pack Finix 3 0422F  1-12U2KT          TV Hogar HD Digital   

   RENTA_CABLE ID_INTERNET   NOMBRE_INTERNET  RENTA_INTERNET  ID_FONO  \
0        23564    1-10KSYE  500M Residencial           21514 

In [26]:

# Condiciones del bundle
conditions_bundle = [
    (~df['ID_CABLE'].isnull()) & (df['ID_INTERNET'].isnull()) & (df['ID_FONO'].isnull()),
    (~df['ID_CABLE'].isnull()) & (df['ID_INTERNET'].isnull()) & (~df['ID_FONO'].isnull()),
    (~df['ID_CABLE'].isnull()) & (~df['ID_INTERNET'].isnull()) & (~df['ID_FONO'].isnull()),
    (~df['ID_CABLE'].isnull()) & (~df['ID_INTERNET'].isnull()) & (df['ID_FONO'].isnull()),
    (df['ID_CABLE'].isnull()) & (df['ID_INTERNET'].isnull()) & (~df['ID_FONO'].isnull()),
    (df['ID_CABLE'].isnull()) & (~df['ID_INTERNET'].isnull()) & (~df['ID_FONO'].isnull()),
    (df['ID_CABLE'].isnull()) & (~df['ID_INTERNET'].isnull()) & (df['ID_FONO'].isnull())
]

# Crear la columna 'BUNDLE' utilizando np.select
choices_bundle = ['C', 'C + F', '3PLAY', 'C + I', 'F', 'I + F', 'I']
df['BUNDLE'] = np.select(conditions_bundle, choices_bundle, default='-1')

# Guardar el DataFrame modificado
ruta_salida = r"C:\Users\wduran\CLAROCHILE\Nicoll Constanza Aguirre Diaz - dcabello\Dashboards Nueva Oferta\INPUTS\Tablas E\20231031_StockTotal_con_bundle.csv"
df.to_csv(ruta_salida, index=False, encoding='latin-1', sep=';')

# Imprimir las primeras filas del DataFrame resultante
print(df.head())



      FECHA RUT_PERSONA  IDEN_VIVIENDA   NOMBRE_CTA_SERV ID_CABLE  \
0  20231031  12941753-6       19844292  12941753-6-S-001  1-5M4L6   
1  20231031   7860335-6        2963044   7860335-6-S-001  1-5M4L6   
2  20231031  19364705-7       19273347  19364705-7-S-006  1-5M4L6   
3  20231031   3360967-1       19468230   3360967-1-S-001  1-KT59J   
4  20231031   7958748-6       19521216   7958748-6-S-001  1-ANC1R   

                NOMBRE_PROMOCION  ID_PROMO                 NOMBRE_CABLE  \
0               VTR Plus 3 1122F  1-13AOTW           TV Full HD Digital   
1               VTR Plus 3 1122F  1-13AOTW           TV Full HD Digital   
2               VTR Plus 2 0822F  1-135UY4           TV Full HD Digital   
3  Triplepack Negocios Mas 1122F  1-13AQMH  TV Negocios Full HD Digital   
4             Pack Finix 3 0422F  1-12U2KT          TV Hogar HD Digital   

   RENTA_CABLE ID_INTERNET   NOMBRE_INTERNET  RENTA_INTERNET  ID_FONO  \
0        23564    1-10KSYE  500M Residencial           21514 

In [30]:
#Conteo de Bundle 
import pandas as pd

# Ruta de tu base de datos
ruta_base_datos = r"C:\Users\wduran\CLAROCHILE\Nicoll Constanza Aguirre Diaz - dcabello\Dashboards Nueva Oferta\INPUTS\Tablas E\20231031_StockTotal_con_bundle.csv"
# Leer la base de datos con el separador adecuado (coma)
df = pd.read_csv(ruta_base_datos, encoding='latin-1', sep=',')

# Conteo del campo 'BUNDLE'
conteo_bundle = df['BUNDLE'].value_counts()

# Mostrar el resultado
print(conteo_bundle)


KeyError: 'BUNDLE'